In [11]:
# ============================================================================
# NOTEBOOK: Limpieza y Transformación Dataset 1 - Encuesta Actividades Turísticas Madrid
# Autor: Alejandro Gerena
# Objetivo: Aplicar limpieza de datos según problemas identificados y 
#           exportar a formato JSON estructurado
# ============================================================================

import pandas as pd
import numpy as np
import json
from datetime import datetime
import re
import os
import csv

In [12]:
# ============================================================================
# ETAPA 1: CARGA Y EXPLORACIÓN INICIAL
# ============================================================================

def cargar_dataset(filename: str) -> pd.DataFrame | None:
    try:
        base_dir = os.getcwd()
        file_path = os.path.join(base_dir, "..", "data", filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"El archivo {file_path} no existe.")

        ext = os.path.splitext(filename)[1].lower()

        if ext == ".csv":
            # Intentar abrir con UTF-8, si falla usar latin-1
            for encoding in ["utf-8", "latin-1"]:
                try:
                    with open(file_path, 'r', encoding=encoding) as f:
                        sample = f.read(2048)
                        sniffer = csv.Sniffer()
                        dialect = sniffer.sniff(sample)
                        detected_sep = dialect.delimiter

                    data = pd.read_csv(file_path, encoding=encoding, sep=detected_sep)
                    break  # Si carga correctamente, salir del bucle
                except UnicodeDecodeError:
                    continue
            else:
                raise UnicodeDecodeError("No se pudo decodificar el archivo con utf-8 ni latin-1")

        elif ext in [".xls", ".xlsx"]:
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Formato de archivo no soportado. Use CSV o Excel (.xls, .xlsx).")

        print(f">>> El archivo '{filename}' se cargó correctamente. Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
        return data

    except FileNotFoundError as e:
        print(e)
    except pd.errors.ParserError as e:
        print(f"Error de formato: {e}")
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

    return None


def explorar_dataset(df):
    """
    Muestra información básica del dataset
    """
    print("\n" + "="*80)
    print("EXPLORACIÓN INICIAL DEL DATASET")
    print("="*80)
    
    print("\n1. Primeras filas:")
    display(df.head(3))
    
    print("\n2. Información de columnas:")
    print(df.info())
    
    print("\n3. Valores nulos por columna:")
    print(df.isnull().sum())
    
    print("\n4. Valores únicos en columnas clave:")
    for col in ['flightNumber', 'plane', 'dep_airport_code', 'arr_airport_code']:
        if col in df.columns:
            print(f"  - {col}: {df[col].nunique()} valores únicos")
    
    return df

In [13]:
# Ejecutar carga y exploración
# Cargar información de vuelos
df_raw = cargar_dataset("EstudioAtencionVisitante2024.xlsx")
df_raw = explorar_dataset(df_raw)

>>> El archivo 'EstudioAtencionVisitante2024.xlsx' se cargó correctamente. Filas: 998, Columnas: 43

EXPLORACIÓN INICIAL DEL DATASET

1. Primeras filas:


,Time End,Sexo,País,"En caso de ser Español, ¿cuál es su provincia de procedencia?",Con quién viaja (sólo una opción),Duración de la estancia en Madrid,Con quién viaja (sólo una opción) - Observaciones,Número de informadores turísticos que están atendiendo,¿Ha buscado información sobre la ciudad de Madrid antes de venir? ¿A través de qué medio? - Observaciones,Oferta de idiomas,...,¿Cómo ha localizado los SAIT? - Observaciones,¿Cuál es el motivo de su visita a la Ciudad de Madrid?,Claridad del lenguaje utilizado,Adecuación de los folletos a la información solicitada,Distribución del espacio (sólo Centros de Turismo,Satisfacción con el Horario de atención al público,¿Qué instalaciones del Servicio de Atención e Información Turística ha visitado? (Seleccione todas las opciones que correspondan),¿ En qué medio de transporte ha llegado a Madrid?,"Aspecto del personal de atención (vestimenta, aseo, identificación)",Señalización informativa
0,2024-03-20,Hombre,Brasil,NaN,En pareja,3 noches,NaN,8,NaN,8,...,NaN,"Ocio, recreo y vacaciones",8.0,8,NS/NC,7,PIT del Aeropuerto T-2,Avión,8,6
1,2024-03-20,Hombre,Alemania,NaN,En familia,3 noches,NaN,9,NaN,7,...,NaN,"Ocio, recreo y vacaciones",10.0,10,10,10,PIT de Atocha,Avión,10,9
2,2024-03-20,Mujer,España,Valencia,En pareja,3 noches,NaN,5,rrss,10,...,NaN,"Ocio, recreo y vacaciones",10.0,10,NS/NC,10,PIT de Recoletos-Colón,Tren,NS/NC,9



2. Información de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 43 columns):
 #   Column                                                                                                                             Non-Null Count  Dtype         
---  ------                                                                                                                             --------------  -----         
 0   Time End                                                                                                                           998 non-null    datetime64[ns]
 1   Sexo                                                                                                                               998 non-null    object        
 2   País                                                                                                                               998 non-null    object        
 3   En caso de ser Español, ¿c

In [14]:
# ============================================================================
# ETAPA 2: LIMPIEZA - PROBLEMA 1 (Nombres de columnas largos y poco manejables)
# ============================================================================

def detectar_problema_1(df):
    """
    Detecta columnas con nombres largos, caracteres especiales, espacios
    """
    print("\n" + "="*80)
    print("PROBLEMA 1: Detección de nombres de columnas problemáticos")
    print("="*80)
    
    problemas = []
    
    for col in df.columns:
        issues = []
        
        if len(col) > 50:
            issues.append(f"muy largo ({len(col)} caracteres)")
        
        if ' ' in col:
            issues.append("contiene espacios")
        
        if any(c in col for c in ['¿', '?', '(', ')', '[', ']', 'á', 'é', 'í', 'ó', 'ú', 'ñ']):
            issues.append("caracteres especiales/tildes")
        
        if issues:
            problemas.append((col, issues))
    
    print(f"\n  Total de columnas con problemas: {len(problemas)}/{len(df.columns)}")
    print("\n  Ejemplos:")
    for col, issues in problemas[:3]:
        print(f"    - '{col[:60]}...'")
        print(f"      Problemas: {', '.join(issues)}")
    
    return problemas


def normalizar_nombre_columna(nombre):
    """
    Normaliza un nombre de columna a snake_case sin caracteres especiales
    """
    # Convertir a minúsculas
    nombre = nombre.lower()
    
    # Reemplazar tildes
    tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ñ': 'n'}
    for tilde, sin_tilde in tildes.items():
        nombre = nombre.replace(tilde, sin_tilde)
    
    # Eliminar signos de interrogación y paréntesis
    nombre = re.sub(r'[¿?()[\]]', '', nombre)
    
    # Reemplazar espacios y guiones por underscore
    nombre = re.sub(r'[\s\-/]+', '_', nombre)
    
    # Eliminar caracteres especiales restantes
    nombre = re.sub(r'[^a-z0-9_]', '', nombre)
    
    # Eliminar underscores múltiples
    nombre = re.sub(r'_+', '_', nombre)
    
    # Eliminar underscores al inicio y final
    nombre = nombre.strip('_')
    
    # Acortar si es muy largo (máximo 80 caracteres)
    if len(nombre) > 80:
        nombre = nombre[:80].rstrip('_')
    
    return nombre


def limpiar_problema_1(df):
    """
    Renombra todas las columnas a formato snake_case normalizado
    """
    df_clean = df.copy()
    
    # Crear mapeo de nombres antiguos a nuevos
    mapeo = {}
    nombres_usados = set()
    
    for col in df_clean.columns:
        nuevo_nombre = normalizar_nombre_columna(col)
        
        # Evitar duplicados añadiendo sufijo numérico si es necesario
        nombre_base = nuevo_nombre
        contador = 1
        while nuevo_nombre in nombres_usados:
            nuevo_nombre = f"{nombre_base}_{contador}"
            contador += 1
        
        mapeo[col] = nuevo_nombre
        nombres_usados.add(nuevo_nombre)
    
    # Aplicar renombrado
    df_clean.rename(columns=mapeo, inplace=True)
    
    print(f"\n✓ {len(mapeo)} columnas renombradas a formato snake_case")
    print("\n  Ejemplos de transformación:")
    for old, new in list(mapeo.items())[:3]:
        print(f"    '{old[:50]}...' → '{new}'")
    
    return df_clean, mapeo


def validar_problema_1(df_limpio):
    """
    Valida que los nombres de columnas sean válidos
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 1")
    print("-"*80)
    
    problemas = 0
    
    for col in df_limpio.columns:
        if ' ' in col or any(c in col for c in ['¿', '?', '(', ')', 'á', 'é', 'í', 'ó', 'ú']):
            problemas += 1
    
    print(f"  Columnas con caracteres problemáticos: {problemas}")
    print(f"  Longitud máxima de nombre: {max(len(c) for c in df_limpio.columns)}")
    
    if problemas == 0:
        print("  ✓ VALIDACIÓN EXITOSA: Todos los nombres normalizados")
        return True
    else:
        print("  ✗ VALIDACIÓN FALLIDA: Aún hay nombres problemáticos")
        return False


# Ejecutar limpieza Problema 1
detectar_problema_1(df_raw)
df_step1, mapeo_columnas = limpiar_problema_1(df_raw)
validar_problema_1(df_step1)


PROBLEMA 1: Detección de nombres de columnas problemáticos

  Total de columnas con problemas: 40/43

  Ejemplos:
    - 'Time End...'
      Problemas: contiene espacios
    - 'País...'
      Problemas: caracteres especiales/tildes
    - 'En caso de ser Español, ¿cuál  es su provincia de procedenci...'
      Problemas: muy largo (62 caracteres), contiene espacios, caracteres especiales/tildes

✓ 43 columnas renombradas a formato snake_case

  Ejemplos de transformación:
    'Time End...' → 'time_end'
    'Sexo...' → 'sexo'
    'País...' → 'pais'

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 1
--------------------------------------------------------------------------------
  Columnas con caracteres problemáticos: 0
  Longitud máxima de nombre: 81
  ✓ VALIDACIÓN EXITOSA: Todos los nombres normalizados


True

In [15]:
# ============================================================================
# ETAPA 3: LIMPIEZA - PROBLEMA 2 (Mezcla de tipos en variables numéricas)
# ============================================================================

def detectar_problema_2(df):
    """
    Detecta columnas que deberían ser numéricas pero contienen texto
    """
    print("\n" + "="*80)
    print("PROBLEMA 2: Detección de mezcla de tipos en variables numéricas")
    print("="*80)
    
    # Identificar columnas de satisfacción (suelen contener "satisfaccion" o números 1-10)
    columnas_sospechosas = []
    
    for col in df.columns:
        if df[col].dtype == 'object':  # Es texto
            # Intentar detectar si debería ser numérica
            valores_unicos = df[col].dropna().unique()
            
            # Verificar si hay mezcla de números y texto
            tiene_numeros = any(str(v).replace('.', '').replace(',', '').isdigit() 
                               for v in valores_unicos if pd.notna(v))
            tiene_texto_no_numerico = any(not str(v).replace('.', '').replace(',', '').isdigit() 
                                          for v in valores_unicos if pd.notna(v) and str(v).strip())
            
            if tiene_numeros and tiene_texto_no_numerico:
                columnas_sospechosas.append((col, valores_unicos))
    
    print(f"\n  Columnas con mezcla de tipos: {len(columnas_sospechosas)}")
    
    for col, valores in columnas_sospechosas[:3]:
        print(f"\n  Columna: '{col}'")
        print(f"    Valores únicos: {valores[:10]}")
    
    return columnas_sospechosas


def limpiar_problema_2(df):
    """
    Convierte columnas de satisfacción a numéricas, 
    reemplazando valores no numéricos por NaN
    """
    df_clean = df.copy()
    
    # Identificar columnas que parecen de satisfacción o numéricas
    columnas_a_limpiar = []
    
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            # Intentar conversión a numérico
            muestra = df_clean[col].dropna().head(20)
            
            # Si la mayoría son números, es candidata
            numeros = sum(1 for v in muestra if str(v).replace('.', '').replace(',', '').isdigit())
            
            if numeros > len(muestra) * 0.5:  # Más del 50% son números
                columnas_a_limpiar.append(col)
    
    print(f"\n  Columnas identificadas para limpieza numérica: {len(columnas_a_limpiar)}")
    
    # Limpiar cada columna
    for col in columnas_a_limpiar:
        # Reemplazar valores comunes no numéricos
        df_clean[col] = df_clean[col].replace({
            'NS/NC': np.nan,
            'NS': np.nan,
            'NC': np.nan,
            'No aplica': np.nan,
            'N/A': np.nan,
            '-': np.nan,
            '': np.nan
        })
        
        # Convertir a numérico (coerce convierte errores a NaN)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    print(f"✓ {len(columnas_a_limpiar)} columnas convertidas a tipo numérico")
    
    return df_clean, columnas_a_limpiar


def validar_problema_2(df_limpio, columnas_limpiadas):
    """
    Valida que las columnas sean numéricas y no contengan texto
    """
    print("\n" + "-"*80)
    print("VALIDACIÓN PROBLEMA 2")
    print("-"*80)
    
    todo_ok = True
    
    for col in columnas_limpiadas:
        if df_limpio[col].dtype not in ['int64', 'float64']:
            print(f"  ✗ {col}: tipo {df_limpio[col].dtype} (debería ser numérico)")
            todo_ok = False
        else:
            valores_no_nulos = df_limpio[col].dropna()
            print(f"  ✓ {col}: tipo {df_limpio[col].dtype}, rango [{valores_no_nulos.min():.1f}, {valores_no_nulos.max():.1f}]")
    
    if todo_ok:
        print("\n  ✓ VALIDACIÓN EXITOSA: Todas las columnas son numéricas")
    else:
        print("\n  ✗ VALIDACIÓN FALLIDA: Algunas columnas no son numéricas")
    
    return todo_ok


# Ejecutar limpieza Problema 2
detectar_problema_2(df_step1)
df_step2, columnas_numericas = limpiar_problema_2(df_step1)
validar_problema_2(df_step2, columnas_numericas)


PROBLEMA 2: Detección de mezcla de tipos en variables numéricas

  Columnas con mezcla de tipos: 22

  Columna: 'numero_de_informadores_turisticos_que_estan_atendiendo'
    Valores únicos: [8 9 5 6 10 7 1 3 'NS/NC' 4]

  Columna: 'oferta_de_idiomas'
    Valores únicos: [8 7 10 'NS/NC' 9 5 1 4 3 6]

  Columna: 'satisfaccion_con_la_infromacion_facilitada_sobre_visitas_guiadas'
    Valores únicos: [8 'NS/NC' 10 9 4 7 5 6 2 3]

  Columnas identificadas para limpieza numérica: 21
✓ 21 columnas convertidas a tipo numérico

--------------------------------------------------------------------------------
VALIDACIÓN PROBLEMA 2
--------------------------------------------------------------------------------
  ✓ numero_de_informadores_turisticos_que_estan_atendiendo: tipo float64, rango [1.0, 10.0]
  ✓ oferta_de_idiomas: tipo float64, rango [1.0, 10.0]
  ✓ conocimiento_del_idioma_utilizado_por_parte_del_personal: tipo float64, rango [2.0, 10.0]
  ✓ satisfaccion_con_la_informacion_facilitada: tip

/tmp/ipykernel_1860/1314340522.py:65: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean[col] = df_clean[col].replace({


True

In [16]:
# ============================================================================
# ETAPA 4: LIMPIEZAS ADICIONALES
# ============================================================================

def limpiezas_adicionales(df):
    """
    Aplica limpiezas adicionales para mejorar calidad
    """
    print("\n" + "="*80)
    print("LIMPIEZAS ADICIONALES")
    print("="*80)
    
    df_clean = df.copy()
    
    # 1. Eliminar filas completamente vacías
    print("\n1. Eliminando filas completamente vacías...")
    filas_antes = len(df_clean)
    df_clean.dropna(how='all', inplace=True)
    filas_eliminadas = filas_antes - len(df_clean)
    print(f"   Filas eliminadas: {filas_eliminadas}")
    
    # 2. Eliminar columnas completamente vacías
    print("2. Eliminando columnas completamente vacías...")
    cols_antes = len(df_clean.columns)
    df_clean.dropna(axis=1, how='all', inplace=True)
    cols_eliminadas = cols_antes - len(df_clean.columns)
    print(f"   Columnas eliminadas: {cols_eliminadas}")
    
    # 3. Limpiar espacios en blanco en columnas de texto
    print("3. Limpiando espacios en blanco en columnas de texto...")
    for col in df_clean.select_dtypes(include=['object']).columns:
        df_clean[col] = df_clean[col].str.strip() if df_clean[col].dtype == 'object' else df_clean[col]
    
    # 4. Estandarizar valores de texto comunes
    print("4. Estandarizando valores de texto...")
    for col in df_clean.select_dtypes(include=['object']).columns:
        df_clean[col] = df_clean[col].replace({
            'Si': 'Sí',
            'si': 'Sí',
            'SI': 'Sí',
            'No': 'No',
            'NO': 'No'
        })
    
    print("\n✓ Limpiezas adicionales completadas")
    return df_clean


# Ejecutar limpiezas adicionales
df_final = limpiezas_adicionales(df_step2)


LIMPIEZAS ADICIONALES

1. Eliminando filas completamente vacías...
   Filas eliminadas: 0
2. Eliminando columnas completamente vacías...
   Columnas eliminadas: 1
3. Limpiando espacios en blanco en columnas de texto...
4. Estandarizando valores de texto...

✓ Limpiezas adicionales completadas


In [17]:
# ============================================================================
# ETAPA 5: RESUMEN FINAL DE CALIDAD
# ============================================================================

def generar_resumen_calidad(df_original, df_final):
    """
    Genera un resumen comparativo de la calidad de datos
    """
    print("\n" + "="*80)
    print("RESUMEN FINAL DE CALIDAD DE DATOS")
    print("="*80)
    
    print(f"\n1. Dimensiones:")
    print(f"   Original: {df_original.shape[0]} filas × {df_original.shape[1]} columnas")
    print(f"   Final:    {df_final.shape[0]} filas × {df_final.shape[1]} columnas")
    
    print(f"\n2. Valores nulos:")
    print(f"   Original: {df_original.isnull().sum().sum()} nulos")
    print(f"   Final:    {df_final.isnull().sum().sum()} nulos")
    
    print(f"\n3. Tipos de datos:")
    print(f"   Columnas numéricas: {len(df_final.select_dtypes(include=['int64', 'float64']).columns)}")
    print(f"   Columnas de texto: {len(df_final.select_dtypes(include=['object']).columns)}")
    
    print("\n✓ Dataset limpio y listo para exportación")


# Generar resumen
generar_resumen_calidad(df_raw, df_final)


RESUMEN FINAL DE CALIDAD DE DATOS

1. Dimensiones:
   Original: 998 filas × 43 columnas
   Final:    998 filas × 42 columnas

2. Valores nulos:
   Original: 6147 nulos
   Final:    7158 nulos

3. Tipos de datos:
   Columnas numéricas: 23
   Columnas de texto: 18

✓ Dataset limpio y listo para exportación


In [18]:
# ============================================================================
# ETAPA 6: EXPORTACIÓN A JSON
# ============================================================================

def safe_value(v):
    """
    Convierte valores NaN de pandas a None (que será null en JSON).
    Convierte Timestamp a string ISO 8601.
    Convierte numpy types a tipos nativos de Python.
    """
    if pd.isna(v):
        return None
    
    # Convertir Timestamp de pandas a string ISO 8601
    if isinstance(v, pd.Timestamp):
        return v.strftime('%Y-%m-%dT%H:%M:%S')
    
    # Convertir numpy types a tipos nativos de Python
    if isinstance(v, (np.integer, )):
        return int(v)
    if isinstance(v, (np.floating, )):
        return float(v)
    
    # Convertir datetime de Python a string ISO 8601
    if isinstance(v, datetime):
        return v.strftime('%Y-%m-%dT%H:%M:%S')
    
    return v


def transformar_a_json_estructura(df):
    """
    Transforma el DataFrame limpio a estructura JSON para encuestas
    
    Returns:
        Lista de diccionarios con la estructura JSON de encuestas
    """
    encuestas_json = []
    
    for idx, row in df.iterrows():
        # Crear documento base
        encuesta = {
            "encuesta_id": safe_value(idx + 1),  # ID secuencial
            "timestamp": datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
            "datos": {}
        }
        
        # Añadir todos los campos del DataFrame
        for col in df.columns:
            encuesta["datos"][col] = safe_value(row[col])
        
        encuestas_json.append(encuesta)
    
    return encuestas_json


def exportar_json(encuestas_json, ruta_salida='encuestas_limpio.json', num_documentos=None, indent=2):
    """
    Exporta la lista de encuestas a un archivo JSON
    
    Args:
        encuestas_json: lista de diccionarios con estructura de encuestas
        ruta_salida: ruta del archivo JSON de salida
        num_documentos: número de documentos a exportar. Si es None, exporta todos.
        indent: nivel de indentación para legibilidad
    """
    # Determinar cuántos documentos exportar
    if num_documentos is None:
        documentos_a_exportar = encuestas_json
        total_exportado = len(encuestas_json)
    else:
        documentos_a_exportar = encuestas_json[:num_documentos]
        total_exportado = min(num_documentos, len(encuestas_json))
    
    # Exportar a JSON
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        json.dump(documentos_a_exportar, f, ensure_ascii=False, indent=indent)
    
    print(f"\n✓ Archivo JSON exportado: {ruta_salida}")
    print(f"  Total de encuestas exportadas: {total_exportado}")
    print(f"  Total de encuestas disponibles: {len(encuestas_json)}")
    
    if num_documentos and num_documentos < len(encuestas_json):
        print(f"  ⚠ Se exportaron solo los primeros {num_documentos} documentos")
    
    tamanio_kb = round(len(json.dumps(documentos_a_exportar)) / 1024, 2)
    print(f"  Tamaño del archivo: {tamanio_kb} KB")


def mostrar_ejemplo_json(encuestas_json, num_ejemplos=1):
    """
    Muestra ejemplos del JSON generado
    """
    print("\n" + "="*80)
    print(f"EJEMPLOS DE DOCUMENTOS JSON GENERADOS (primeros {num_ejemplos})")
    print("="*80)
    
    for i, encuesta in enumerate(encuestas_json[:num_ejemplos], 1):
        print(f"\n--- Encuesta {i} ---")
        print(json.dumps(encuesta, ensure_ascii=False, indent=2))

# Ejecutar transformación y exportación
encuestas_json = transformar_a_json_estructura(df_final)
mostrar_ejemplo_json(encuestas_json, num_ejemplos=1)


EJEMPLOS DE DOCUMENTOS JSON GENERADOS (primeros 1)

--- Encuesta 1 ---
{
  "encuesta_id": 1,
  "timestamp": "2025-11-28T01:11:43",
  "datos": {
    "time_end": "2024-03-20T00:00:00",
    "sexo": "Hombre",
    "pais": "Brasil",
    "en_caso_de_ser_espanol_cual_es_su_provincia_de_procedencia": null,
    "con_quien_viaja_solo_una_opcion": "En pareja",
    "duracion_de_la_estancia_en_madrid": "3 noches",
    "con_quien_viaja_solo_una_opcion_observaciones": null,
    "numero_de_informadores_turisticos_que_estan_atendiendo": 8.0,
    "ha_buscado_informacion_sobre_la_ciudad_de_madrid_antes_de_venir_a_traves_de_que": null,
    "oferta_de_idiomas": 8.0,
    "satisfaccion_con_la_infromacion_facilitada_sobre_visitas_guiadas": null,
    "como_ha_localizado_los_sait": "Los ha encontrado en su recorrido",
    "cual_es_el_motivo_de_su_visita_a_la_ciudad_de_madrid_observaciones": null,
    "trato_y_amabilidad": 8.0,
    "conocimiento_del_idioma_utilizado_por_parte_del_personal": 8.0,
    "satisfaccio

In [19]:
# Exportar todos los documentos
#exportar_json(encuestas_json, ruta_salida='dataset1_encuestas_limpio.json')
# O exportar solo una muestra para pruebas
exportar_json(encuestas_json, ruta_salida='dataset1_encuestas_muestra_100.json', num_documentos=100)


✓ Archivo JSON exportado: dataset1_encuestas_muestra_100.json
  Total de encuestas exportadas: 100
  Total de encuestas disponibles: 998
  ⚠ Se exportaron solo los primeros 100 documentos
  Tamaño del archivo: 227.1 KB


In [20]:
# ============================================================================
# FIN DEL NOTEBOOK
# ============================================================================

print("\n" + "="*80)
print("PROCESO COMPLETADO EXITOSAMENTE")
print("="*80)
print("\nArchivo generado: dataset1_encuestas_limpio.json")
print("El dataset está listo para la entrega.")


PROCESO COMPLETADO EXITOSAMENTE

Archivo generado: dataset1_encuestas_limpio.json
El dataset está listo para la entrega.
